## Welcome to the data cleaning phase of this project!
We currently have a folder `table-data` with a bunch of dirty json files. The goal of this notebook is to illustrate the process of data cleaning by subsequently transforming the dirty json tables into dataframes with tidied up data, ready to be stored in a database and then analyzed!

There are 6 different tables for every printing log:
* Overview
* Bioinks
* Material Settings
* Printer Setup
* Hardware Setup
* Printing Log

The cleaning steps will refer to the codes B1 - B5 and represent the listed functions stated in the theoretical part of the thesis. B1 was already achieved in the file `structureTables.js`. The starting point of this notebook is a folder containing structured json files with added `id` field referencing the corresponding experiment.

First, let's import some libraries

In [961]:
import os
import json
import pandas as pd
import numpy as np

# Load mappings from JSON file
with open('mappings.json', 'r') as file:
    mappings = json.load(file)

Next, let's define a function which is used throughout this notebook to access the mapping-dictionary in `mappings.json`.

In [ ]:
# Helper function to find the key for a given value
def find_key(mapping, value):
    value = value.strip().lower()  # Strip whitespaces and convert to lower case for comparison
    for key, values in mapping.items():
        if any(val.lower() in value for val in values):  # Compare lower case values
            return key
    return "other"

# Create Dataframes from Json (B2)

Let's iterate through each JSON file within the 'overview' category, aggregating their contents into a single comprehensive dataframe.

## Overview Table

In [962]:
data = []
folder_path = 'table-data-cleaned'
# Iterate through each subfolder
for index, subfolder in enumerate(os.listdir(folder_path)):
    subfolder_path = os.path.join(folder_path, subfolder)
    if os.path.isdir(subfolder_path):
        # Check if "bioinks.json" exists in the subfolder
        jsonPath = os.path.join(subfolder_path, "overview" + ".json")
        if os.path.exists(jsonPath):
            # Read the content of "bioinks.json" and append it to the list
            with open(jsonPath, 'r') as f:
                content = json.load(f)
                data.extend(content)
                
# Convert the list to a pandas DataFrame
overviewDf = pd.DataFrame(data)

overviewDf.to_csv("data-frames-raw/" + "overview" + '.csv', index=False)

## Bioinks Table

In [963]:
data = []
folder_path = 'table-data-cleaned'
# Iterate through each subfolder
for index, subfolder in enumerate(os.listdir(folder_path)):
    subfolder_path = os.path.join(folder_path, subfolder)
    if os.path.isdir(subfolder_path):
        # Check if "bioinks.json" exists in the subfolder
        jsonPath = os.path.join(subfolder_path, "bioInks" + ".json")
        if os.path.exists(jsonPath):
            # Read the content of "bioinks.json" and append it to the list
            with open(jsonPath, 'r') as f:
                content = json.load(f)
                data.extend(content)
                
# Convert the list to a pandas DataFrame
bioInksDf = pd.DataFrame(data)
bioInksDf.to_csv("data-frames-raw/" + "bioInks" + '.csv', index=False)

## Hardware Setup Table

In [964]:
data = []
folder_path = 'table-data-cleaned'
# Iterate through each subfolder
for index, subfolder in enumerate(os.listdir(folder_path)):
    subfolder_path = os.path.join(folder_path, subfolder)
    if os.path.isdir(subfolder_path):
        # Check if "bioinks.json" exists in the subfolder
        jsonPath = os.path.join(subfolder_path, "hardwareSetup" + ".json")
        if os.path.exists(jsonPath):
            # Read the content of "bioinks.json" and append it to the list
            with open(jsonPath, 'r') as f:
                content = json.load(f)
                data.extend(content)
                
# Convert the list to a pandas DataFrame
hardwareSetupDf = pd.DataFrame(data)

hardwareSetupDf.to_csv("data-frames-raw/" + "hardwareSetup" + '.csv', index=False)

## Printer Setup Table

In [965]:
data = []
folder_path = 'table-data-cleaned'
# Iterate through each subfolder
for index, subfolder in enumerate(os.listdir(folder_path)):
    subfolder_path = os.path.join(folder_path, subfolder)
    if os.path.isdir(subfolder_path):
        # Check if "bioinks.json" exists in the subfolder
        jsonPath = os.path.join(subfolder_path, "printerSetup" + ".json")
        if os.path.exists(jsonPath):
            # Read the content of "bioinks.json" and append it to the list
            with open(jsonPath, 'r') as f:
                content = json.load(f)
                data.extend(content)
                
# Convert the list to a pandas DataFrame
printerSetupDf = pd.DataFrame(data)

printerSetupDf.to_csv("data-frames-raw/" + "printerSetup" + '.csv', index=False)

The printerSetup table only contains filenames and has a lot of missing values. Thus, it is considered irrelevant for this analysis and can be discarded.

## Material Settings Table

In [966]:
data = []
folder_path = 'table-data-cleaned'
# Iterate through each subfolder
for index, subfolder in enumerate(os.listdir(folder_path)):
    subfolder_path = os.path.join(folder_path, subfolder)
    if os.path.isdir(subfolder_path):
        # Check if "bioinks.json" exists in the subfolder
        jsonPath = os.path.join(subfolder_path, "materialSettings" + ".json")
        if os.path.exists(jsonPath):
            # Read the content of "bioinks.json" and append it to the list
            with open(jsonPath, 'r') as f:
                content = json.load(f)
                data.extend(content)
                
# Convert the list to a pandas DataFrame
materialSettingDf = pd.DataFrame(data)

materialSettingDf.to_csv("data-frames-raw/" + "materialSettings" + '.csv', index=False)

## Printing Log Table

In [967]:
data = []
folder_path = 'table-data-cleaned'
# Iterate through each subfolder
for index, subfolder in enumerate(os.listdir(folder_path)):
    subfolder_path = os.path.join(folder_path, subfolder)
    if os.path.isdir(subfolder_path):
        # Check if "bioinks.json" exists in the subfolder
        jsonPath = os.path.join(subfolder_path, "printingLog" + ".json")
        if os.path.exists(jsonPath):
            # Read the content of "bioinks.json" and append it to the list
            with open(jsonPath, 'r') as f:
                content = json.load(f)
                data.extend(content)
                
# Convert the list to a pandas DataFrame
printingLogDf = pd.DataFrame(data)

printingLogDf.to_csv("data-frames-raw/" + "printingLog" + '.csv', index=False)

# Clean Columns (B3)

## Overview Table

This table doesn't have unnecessary columns but the column 'folderName' contains date information so a new column 'date' is needed. Furthermore, the column names should be converted to lowercase.
Also, the entries in the column 'Log no.' seem to be subset of the entries in the column 'folderName' and thus can be dropped to reduce redundancy.

In [968]:
# Create new columns
overviewDf['date'] = None

# Drop column 'Log no.'
overviewDf = overviewDf.drop(columns=['Log no.'])

# Convert column names to lowercase
overviewDf.columns = overviewDf.columns.str.lower()

Save to csv

In [969]:
overviewDf.to_csv("data-frames-cleaned/" + "overview" + '.csv', index=False)

## Bioinks Table

The dataframe has several columns which are mostly empty. Before removing columns, let's look at them, maybe we can map some to similar values.

In [970]:
print(", ".join(bioInksDf.columns))

Name, Polymer, cpolymer [v%], cLAP [wt%], CTartrazine [mM], Solvent, logId, spheres, CIodixanol [v/v], Gelma FIC A29, other, cLAP [v%], CDTT [v%], Color [mg], additives, Additives, Filter [µM], Add, Fluorospheres [dil], Comment, Amount of color(mg/ml), Beads Mio/ml, cDTT [v%], Peptide (g/L)


cLAP [v%] and cLAP [wt%] describe the same thing but are in different units. Let's change that. Same goes for CDTT [v%] and CTartrazine [mM].

In [971]:
import re

bioInksDf['CTartrazine [mM]'] = bioInksDf['CTartrazine [mM]'].combine_first(bioInksDf['CDTT [v%]'])
# only get the values in mM

def extract_mM(value):
    # Check if value is a string
    if isinstance(value, str):
        # Use regular expression to find the pattern
        match = re.search(r'\((\d+)mM\)', value)
        if match:
            # Extract and return the number
            return int(match.group(1))
    # Return NaN or some default value if pattern not found or value is not a string
    return value

# Apply the function to the desired column
bioInksDf['CTartrazine [mM]'] = bioInksDf['CTartrazine [mM]'].apply(extract_mM)

# Write to csv
bioInksDf.to_csv("data-frames-cleaned/" + "bioInks" + '.csv', index=False)

In the following step we will remove the superfluous columns.

In [972]:
rmCols = [
    "spheres" , "CIodixanol [v/v]", "Gelma FIC A29", "other", "cLAP [v%]", "CDTT [v%]", "Color [mg]", "additives", "Additives", "Filter [µM]", 
    "Add", "Fluorospheres [dil]", "Comment", "Amount of color(mg/ml)", "Beads Mio/ml", "cDTT [v%]", "Peptide (g/L)"
]

# Remove the specified columns from the DataFrame
bioInksDf = bioInksDf.drop(columns=rmCols, errors='ignore')
print(f"Columns {rmCols} removed for type {type}.")

# Write to csv
bioInksDf.to_csv("data-frames-cleaned/" + "bioInks" + '.csv', index=False)

Columns ['spheres', 'CIodixanol [v/v]', 'Gelma FIC A29', 'other', 'cLAP [v%]', 'CDTT [v%]', 'Color [mg]', 'additives', 'Additives', 'Filter [µM]', 'Add', 'Fluorospheres [dil]', 'Comment', 'Amount of color(mg/ml)', 'Beads Mio/ml', 'cDTT [v%]', 'Peptide (g/L)'] removed for type <class 'type'>.


The column names need to be converted to lower case and stripped of the white spaces.

In [973]:
# Convert column names to lowercase
bioInksDf.columns = [col.lower() for col in bioInksDf.columns]

bioInksDf.columns = bioInksDf.columns.str.replace(' ', '_').str.replace('[\[\]]', '', regex=True).str.replace('%', '')

# Write to csv
bioInksDf.to_csv("data-frames-cleaned/" + "bioInks" + '.csv', index=False)

## Hardware Table

The hardware table has some columns which need to mapped.
There are three columns which describe the ink reservoir/drying setup. Also, the last column 'Printhead: CB5' can be dropped since it doesn't contain values. After that, convert the column names to lowercase.

In [974]:
rows_to_concat = ['Inkreservoir/Drying', 'Ink reservoir/drying', 'Ink reservoir/Drying']

hardwareSetupDf['inkreservoir/drying'] = hardwareSetupDf.apply(
    lambda row: row['Inkreservoir/Drying'] if pd.notnull(row['Inkreservoir/Drying'])
    else (row['Ink reservoir/drying'] if pd.notnull(row['Ink reservoir/drying'])
    else row['Ink reservoir/Drying']), axis=1)

hardwareSetupDf = hardwareSetupDf.drop(['Inkreservoir/Drying', 'Ink reservoir/drying', 'Ink reservoir/Drying', 'Printhead: CB5'], axis=1)

# Convert column names to lowercase
hardwareSetupDf.columns = hardwareSetupDf.columns.str.lower()

# Write to csv
hardwareSetupDf.to_csv("data-frames-cleaned/" + "hardwareSetup" + '.csv', index=False)


# Material Settings Table

Next, merge the column .JSON with column "Material parameters".

In [975]:
materialSettingDf.loc[materialSettingDf['Material parameters'].isna(), 'Material parameters'] = materialSettingDf['.JSON']

materialSettingDf = materialSettingDf.drop(['.JSON', 'Attempt', 'Material'], axis=1)

# Write to csv
materialSettingDf.to_csv("data-frames-cleaned/" + "materialSettings" + '.csv', index=False)

The column "Material parameters" has json values which need their own columns.

In [976]:
import re

columns_to_initialize = [
    'temperature', 'temperatureTolerance', 'brightness', 'exposureTime', 'zHop',
    'zHopSpeed', 'washTime', 'dabCount', 'washCount', 'projectionDelay'
]
for column in columns_to_initialize:
    materialSettingDf[column] = ""

def processJson(row):
    string = row['Material parameters']
    # Check for substrings that should cause the function to abort and return the string as is
    if "http" in string or "Slice" in string:
        return row
    # Normalize the string by removing any leading/trailing braces and whitespace
    if "{" in string or '"' in string:   
        normalized_str = re.sub(r'^\s*{\s*|\s*}\s*$', '', string)
        pairs = re.findall(r'(?:"(\w+)":\s*([^,}]+))', normalized_str)
    else:
        normalized_str = string
        pairs = re.findall(r'(?:(\w+):\s*([^,}]+))', normalized_str)

    # Process each pair and convert numerical values from string to their appropriate type
    for key, value in pairs:
        # Remove any non-numeric trailing characters from the value
        value = re.sub(r'[^0-9.]+$', '', value.strip())
        # Try converting to integer, if fail, then to float, if fail, keep as string
        try:
            value = int(value)
        except ValueError:
            try:
                value = float(value)
            except ValueError:
                pass  # keep the value as string if it's neither int nor float
        # Assign the value to the row if the key is a column in the DataFrame
        if key in materialSettingDf.columns:
            row[key] = value
            
    return row

materialSettingDf = materialSettingDf.apply(processJson, axis=1)

# Drop useless columns
materialSettingDf = materialSettingDf.drop(["Material parameters", "File name", "File name of material .json", "File name of material .Json"], axis=1)

# Write to csv
materialSettingDf.to_csv("data-frames-cleaned/" + "materialSettings" + '.csv', index=False)

Convert columns to lowercase.

In [977]:
# Convert column names to lowercase
materialSettingDf.columns = materialSettingDf.columns.str.lower()

# Write to csv
materialSettingDf.to_csv("data-frames-cleaned/" + "materialSettings" + '.csv', index=False)

# Printing Log Table

Let's start by looking at the columns and figuring out which columns describe the same values and which columns can be dropped.

In [978]:
print(printingLogDf.columns.tolist())

['Attempt No.', 'Ink', 'layer height [µm]', 'no. bottom layers', 'texp.bottom\xa0 [s]', 'texposure_pos1 [s]', 'texposure_pos2 [s]', 'texposure_pos3 [s]', 'texposure_pos4 [s]', 'zSpeed', 'status', 'comment', 'next steps', 'pictures', 'logId', 'texposure_sec[s]', 'texposure_last [s]', 'Washing/drying process', 'Ink1', 'Ink2', 'washing', 'drying', 'WASH/ DRY technique', 'GLOBAL OFFSET', 'texp.bottom\u202f [s]', 'Texposure_pos1 [s]', 'Texposure_pos2 [s]', 'Texposure_pos3 [s]', 'Texposure_pos4 [s]', 'texp.layer [s]', 'zHop [mm]', 'LAP [%]', 'texp.layer 1 [s]', 'texp.layer 2 [s]', 'texp.layer 3 [s]', 'texp.layer 4 [s]', 'Post-curing time [min]', 'Tart', 'layer height [mm]', 'Base height', 'OFFSET', 'VARIABLE CHANGED', 'texposure [s]', 'Tart [mM]', 'zHop', 'Comment\xa010 s nachbelichtet']


In [979]:
# Drop
printingLogDf = printingLogDf.drop(columns=['OFFSET', 'Tart', 'Tart [mM]', 'Base height', 'zHop', 'Post-curing time [min]', 'VARIABLE CHANGED', 
                                            'Comment 10 s nachbelichtet', 'LAP [%]', 'GLOBAL OFFSET', 'texposure [s]', 'Ink1', 'Ink2', 
                                            'Washing/drying process', 'washing', 'drying', 'WASH/ DRY technique', 'texposure_sec[s]', 
                                            'texposure_last [s]', 'texp.bottom  [s]', 'Texposure_pos2 [s]', 'Texposure_pos3 [s]',
                                            'Texposure_pos4 [s]', 'zHop [mm]', 'layer height [mm]', 'texp.layer 2 [s]', 'texp.layer 3 [s]', 
                                            'texp.layer 4 [s]', 'texp.layer [s]', 'pictures'], axis=1)

# Map texposure_pos1 [s]
printingLogDf['texposure1_s'] = printingLogDf.apply(
    lambda row: row['texposure_pos1 [s]'] if pd.notnull(row['texposure_pos1 [s]'])
    else (row['Texposure_pos1 [s]'] if pd.notnull(row['Texposure_pos1 [s]'])
    else row['texp.layer 1 [s]']), axis=1)

printingLogDf = printingLogDf.drop(['texposure_pos1 [s]', 'Texposure_pos1 [s]', 'texp.layer 1 [s]'], axis=1)
# ink <-
# layerHeight <- 'layer height [µm]', layer height [mm]

# Save to csv
printingLogDf.to_csv("data-frames-cleaned/" + "printingLog" + '.csv', index=False)

# Clean rows (B4)

## Overview Table

The dataframe currently contains some cells, which can be filled from existing content. As mentioned, the column 'date' can be filled with values from 'foldername'. Furthermore 'foldername' can be used to enrich missing values in the 'title' column. Lastly, empty rows can be deleted.

In [980]:
# Adjust the function to handle additional rules
import re
from datetime import datetime

def extract_date(log):
    # Extract the numeric part of the log string
    numeric_part = re.search(r'\d+', log)
    if numeric_part:
        numeric_part = numeric_part.group()
        # If the numeric part ends with '01', '02', '03', '04', '05', '06', '07', which might not be part of the date, remove it
        if numeric_part.endswith(('01', '02', '03', '04', '05', '06', '07')) and len(numeric_part) > 6:
            numeric_part = numeric_part[:-2]
        # Handle different date formats
        if len(numeric_part) == 6:
            # Assume the format is YYMMDD
            date_format = '%y%m%d'
        elif len(numeric_part) == 8:
            # Assume the format is YYYYMMDD
            date_format = '%Y%m%d'
        else:
            return None  # Return None if the format is unrecognized
        # Parse the date
        try:
            return datetime.strptime(numeric_part, date_format).date()
        except ValueError:
            return None  # Return None if the date parsing fails
    return None  # Return None if no numeric part is found

# Apply the adjusted date extraction function
overviewDf['date'] = overviewDf['foldername'].apply(extract_date)

In [981]:
def enrich_title(row):
    if row['title'] == '':
        row['title'] = row['foldername']
    return row

# Apply the function to each row
overviewDf = overviewDf.apply(enrich_title, axis=1)

overviewDf = overviewDf.drop(columns=['foldername'], axis=1)

In [982]:
# Select all columns except 'id' since every row has an entry in that row
columns_to_check = overviewDf.columns.drop('id')

# Remove rows where any of the specified columns have an empty string
overviewDf = overviewDf[~overviewDf[columns_to_check].map(lambda x: x == '').all(axis=1)]

In [983]:
# Save the cleaned DataFrame back to a CSV file
overviewDf.to_csv('data-frames-cleaned/' + "overview" + '.csv', index=False)

## Bioinks

First, let's remove empty lines. First, characters indicating an empty cell are being replaced with an empty string to help streamline the process

In [984]:
# remove empty-indicating characters
rmCharacters = ['-', '•', '/', '%']
bioInksDf = bioInksDf.replace(rmCharacters, '', regex=True)

# Replace empty strings with NaN for consistency
bioInksDf = bioInksDf.replace('', np.nan)

# Select all columns except 'logId'
columns_to_check = bioInksDf.columns.drop('logid')

# Remove rows where any of the specified columns are NaN (or empty after previous operations)
bioInksDf = bioInksDf.dropna(subset=columns_to_check, how='all')

# Write to csv
bioInksDf.to_csv("data-frames-cleaned/" + "bioInks" + '.csv', index=False)

Let's look at the column 'polymer'. We want to map similar bioinks to the same value.

In [985]:
bioInksMappings = mappings["bioInks"]

def map_polymer(polymer):
    polymer = polymer.strip()  # Remove any leading/trailing whitespace
    return find_key(bioInksMappings['polymer'], polymer)
    
def map_solvent(solvent):
    solvent = solvent.strip()  # Remove any leading/trailing whitespace
    return find_key(bioInksMappings['solvent'], solvent)

def removeChars(row):
    polymerString = row['polymer']
    solventString = row['solvent']

    # Check if polymerString is a string, else set to empty string
    if not isinstance(polymerString, str):
        polymerString = ''
    else:
        polymerString = ''.join(filter(lambda x: not x.isdigit(), polymerString))
    
    splitStrings = polymerString.split() if polymerString else []

    # Check if solventString is a string, else set to empty string
    if not isinstance(solventString, str):
        solventString = ''
    
    splitSolventStrings = solventString.split() if solventString else []
    
    # Map each potential ink and remove 'unknown' values
    mapped_inks = [map_polymer(potentialInk) for potentialInk in splitStrings]
    splitStrings = [ink for ink in mapped_inks if ink != 'other']
    row['polymer'] = splitStrings
    
    # Map solvents
    mapped_solvents = [map_solvent(potentialSolvent) for potentialSolvent in splitSolventStrings]
    splitSolventStrings = [solvent for solvent in mapped_solvents if solvent != 'unknown']
    row['solvent'] = splitSolventStrings
             
    return row
    
# column to lowercase
bioInksDf['polymer'] = bioInksDf['polymer'].str.lower()
bioInksDf['solvent'] = bioInksDf['solvent'].str.lower()

bioInksDf = bioInksDf.apply(removeChars, axis=1)

# Write to csv
bioInksDf.to_csv("data-frames-cleaned/" + "bioInks" + '.csv', index=False)

Next, we want to give every double ink in a row its own row.

In [986]:
def scanForTwoInks(row, new_rows):
    polymerArray = row['polymer']
    # Check if cpolymer_v is a string, else set to empty string
    cPolymers = row['cpolymer_v']
    if not isinstance(cPolymers, str):
        cPolymers = ''
    else:
        cPolymers = cPolymers.split()
    solventsArray = row['solvent']
    inkName = row['name']
    if len(polymerArray) > 1 and len(cPolymers) > 1:
        # Ensure that polymerArray and cPolymers have the same length
        # This step is crucial as it ensures that there's a 1:1 mapping between polymers and concentrations
        min_length = min(len(polymerArray), len(cPolymers))
        polymerArray = polymerArray[:min_length]
        cPolymers = cPolymers[:min_length]
        
        for i in range(min_length):
            new_row = row.copy()
            new_row['polymer'] = [polymerArray[i]]
            new_row['cpolymer_v'] = [cPolymers[i]]
            if len(solventsArray) == min_length:
                new_row['solvents'] = solventsArray[i]
            new_row['name'] = inkName
            new_rows.append(new_row)
            # Mark the index of the row to be dropped since it has been expanded into new rows
            index_to_drop.append(row.name)
    elif len(polymerArray) == 2 and len(cPolymers) == 1:
        new_row = row.copy()
        new_row['polymer'] = [polymerArray[0]]
        new_row['cpolymer_v'] = [cPolymers[0]]
        new_row['name'] = inkName
        new_rows.append(new_row)
        # Mark the index of the row to be dropped since it has been expanded into new rows
        index_to_drop.append(row.name)
    else:
        new_rows.append(row)  # If there's only one polymer, keep the row as is
        
new_rows = []  # Initialize an empty list to collect new rows
index_to_drop = []

# Iterate over each row in the DataFrame
for index, row in bioInksDf.iterrows():
    scanForTwoInks(row, new_rows)
    
# Drop the original rows that have been expanded into multiple rows
bioInksDf = bioInksDf.drop(index_to_drop)
    
new_bioInksDf = pd.DataFrame(new_rows)

bioInksDf = pd.concat([bioInksDf, new_bioInksDf], ignore_index=True)

# Write to csv
bioInksDf.to_csv("data-frames-cleaned/" + "bioInks" + '.csv', index=False)

Let's clean up the table some more

In [987]:
def clean_dataframe(df):
    # polymer: Remove brackets and quotes
    df['polymer'] = df['polymer'].str[0]

    # cpolymer_v: Keep only numbers and replace commas with points
    df['cpolymer_v'] = df['cpolymer_v'].str.replace(r'[^0-9,\.]', '', regex=True)
    df['cpolymer_v'] = df['cpolymer_v'].str.replace(',', '.')

    # clap_wt: Remove non-numeric entries and entries with ">"
    df['clap_wt'] = df['clap_wt'].str.replace(r'[^0-9,\.]', '', regex=True)
    df['clap_wt'] = df['clap_wt'].str.replace(',', '.')
    df['clap_wt'] = df['clap_wt'].replace(r'.*?>.*', '', regex=True)

    # ctartrazine_mm: Same cleaning as clap_wt
    df['ctartrazine_mm'] = df['ctartrazine_mm'].str.replace(r'[^0-9,\.]', '', regex=True)
    df['ctartrazine_mm'] = df['ctartrazine_mm'].str.replace(',', '.')
    df['ctartrazine_mm'] = df['ctartrazine_mm'].replace(r'.*?>.*', '', regex=True)

    # Solvent: Take the first entry and remove brackets and quotes
    df['solvent'] = df['solvent'].str[0]

    return df

# Apply the cleaning function to the DataFrame
bioInksDf = clean_dataframe(bioInksDf)

bioInksDf = bioInksDf.drop(columns=['solvents'])

# Write to csv
bioInksDf.to_csv("data-frames-cleaned/" + "bioInks" + '.csv', index=False)

Next, fill missing numbers with the mean

In [988]:
def fill_missing_with_zeros(row):
    for column in ['cpolymer_v', 'clap_wt', 'ctartrazine_mm']:
        if isinstance(row[column], str):
            if row[column] == '':
                row[column] = 0
        try:
            row[column] = float(row[column])
        except ValueError:
            row[column] = 0

    return row

# Apply the function to the DataFrame
bioInksDf = bioInksDf.apply(fill_missing_with_zeros, axis=1)

# Write to csv
bioInksDf.to_csv("data-frames-cleaned/" + "bioInks" + '.csv', index=False)

There are some duplicate rows which need to be removed.

In [989]:
bioInksDf = bioInksDf.drop_duplicates()

# Write to csv
bioInksDf.to_csv("data-frames-cleaned/" + "bioInks" + '.csv', index=False)

The bioinks table looks clean now. ✅

# Hardware Table

First of all, drop rows without content in the relevant columns 'printhead' and 'inkreservoir/drying'.

In [990]:
# Replace specific characters with empty strings
rmCharacters = ['-', '•', '/', '%']
hardwareSetupDf = hardwareSetupDf.replace(rmCharacters, '', regex=True)

# Select all columns except 'logId'
columns_to_check = hardwareSetupDf.columns.drop(['logid', 'position'])

# Remove rows where any of the specified columns have an empty string
hardwareSetupDf = hardwareSetupDf[~hardwareSetupDf[columns_to_check].apply(lambda x: x == '').any(axis=1)]

# Write to csv
hardwareSetupDf.to_csv("data-frames-cleaned/" + "hardwareSetup" + '.csv', index=False)

Now we can start the mapping

In [991]:
hardwareMappings = mappings['hardwareSetup']

def map_values(row):
    
    # Handle NaN values and map printhead
    printhead = str(row['printhead']) if pd.notna(row['printhead']) else ''
    row['printhead'] = find_key(hardwareMappings['printhead'], printhead)

    # Handle NaN values and map inkreservoir/drying
    inkreservoir_drying = str(row['inkreservoir/drying']) if pd.notna(row['inkreservoir/drying']) else ''
    row['inkreservoir/drying'] = find_key(hardwareMappings['inkreservoir_drying'], inkreservoir_drying)

    return row

# Apply the function to the DataFrame
hardwareSetupDf = hardwareSetupDf.apply(map_values, axis=1)

# Write to csv
hardwareSetupDf.to_csv("data-frames-cleaned/" + "hardwareSetup" + '.csv', index=False)

The hardware table looks clean now. ✅

# Material Settings Table

Remove empty lines.

In [992]:
# Replace specific characters with empty strings
rmCharacters = ['-', '•', '/', '%']
materialSettingDf = materialSettingDf.replace(rmCharacters, '', regex=True)

# Select all columns except 'logId'
columns_to_check = materialSettingDf.columns.drop(['logid', 'position'])

# Remove rows where any of the specified columns have an empty string
materialSettingDf = materialSettingDf[~materialSettingDf[columns_to_check].apply(lambda x: x == '').any(axis=1)]

# Write to csv
materialSettingDf.to_csv("data-frames-cleaned/" + "materialSettings" + '.csv', index=False)

Now, a few rows still miss values. Let's fill those with the mean value of the corresponding row.

In [993]:
import numpy as np

columns_to_fill = ['temperature', 'temperaturetolerance', 'brightness', 'exposuretime', 'zhop', 'zhopspeed', 'washtime']

materialSettingDf['exposuretime'] = materialSettingDf['exposuretime'].apply(lambda x: 0 if len(str(x)) > 3 else x)

for column in columns_to_fill:
    materialSettingDf[column] = pd.to_numeric(materialSettingDf[column], errors='coerce')
    

column_means = materialSettingDf[columns_to_fill].mean()

materialSettingDf.fillna(column_means, inplace=True)

for column in columns_to_fill:
    materialSettingDf[column].replace(0, column_means[column], inplace=True)

# Write to csv
materialSettingDf.to_csv("data-frames-cleaned/" + "materialSettings" + '.csv', index=False)

# Printing Log Table

Next, remove empty rows.

In [994]:
# Replace empty strings with NaN
# Remove rows where the 'status' column is NaN or empty
printingLogDf = printingLogDf.dropna(subset=['status'])

# If you also want to consider empty strings as missing values and remove those rows:
printingLogDf = printingLogDf[printingLogDf['status'].astype(bool)]

# Save to csv
printingLogDf.to_csv("data-frames-cleaned/" + "printingLog" + '.csv', index=False)

Next, map the 'status' column

In [995]:
printingLogMappings = mappings["printingLog"]

def map_values(row):
    # Map 'status' column
    row['status'] = find_key(printingLogMappings['status'], row['status'])
    return row
    
# column to lowercase
printingLogDf['status'] = printingLogDf['status'].str.lower()

# Apply the function to the DataFrame
printingLogDf = printingLogDf.apply(map_values, axis=1)

# Save to csv
printingLogDf.to_csv("data-frames-cleaned/" + "printingLog" + '.csv', index=False)


Let's remove some annoying chars.

In [954]:
# remove empty-indicating characters
rmCharacters = ['-', '•', '/']
printingLogDf = printingLogDf.replace(rmCharacters, '', regex=True)

# Write to csv
printingLogDf.to_csv("data-frames-cleaned/" + "printingLog" + '.csv', index=False)

# Convert Data Types (B5)

## Overview Table

Now, let's look at the column "Operator". This column currently includes ambigous values. For example "JV/TL" and "JV,TL" mean the same thing.
The following code cleans thoes ambiguities and puts the operators in an array of strings and thus converts it to a data type which can be processed more easily later on.

In [955]:
import re

def split_operators(operator):
    operator = operator.upper()
    # Remove spaces around delimiters and parentheses
    operator = re.sub(r'\s*([,/&+()])\s*', r'\1', operator)
    # Split on the delimiters
    parts = re.split(r'[,/&+() ]', operator)
    # Remove empty strings and strip whitespace
    parts = [part.strip() for part in parts if part.strip()]
    # Special handling for initials (e.g., 'John Doe' -> 'JD')
    if len(parts) == 1 and ' ' in parts[0]:
        parts = [''.join([name[0] for name in parts[0].split() if name])]
        parts.upper()
    return parts

overviewDf["operators"] = overviewDf["operator"].apply(split_operators)


That looks good, now let's drop the "operator" column.

In [956]:
overviewDf = overviewDf.drop(columns=['operator'], axis=1)

In [957]:
# Save the cleaned DataFrame back to a CSV file
overviewDf.to_csv('data-frames-cleaned/' + "overview" + '.csv', index=False)

The overview table looks clean now. ✅

# Data Storage (C)

Now we can create a SQLite database from our cleaned dataframes. For that, we first need to split up the overview dataframe and create a new one for the 'operators' column.
Also, we will merge materialSettings and hardwareSetup since both dataframes contain slot-related information.

In [958]:
# Expanding the operator_array to a new dataframe
operator_data = [(row['id'], operator) for index, row in overviewDf.iterrows() for operator in row['operators']]
operatorsDf = pd.DataFrame(operator_data, columns=['id', 'operator'])

# Creating the main dataframe without the operator_array column
overviewDf = overviewDf.drop('operators', axis=1)

# Merge all slot related tables
slotSettingsDf = pd.merge(materialSettingDf, hardwareSetupDf, on=['logid', 'position'])

# save the changed dataframes again
overviewDf.to_csv("data-frames-cleaned/" + "overview" + '.csv', index=False)
operatorsDf.to_csv("data-frames-cleaned/" + "operators" + '.csv', index=False)
slotSettingsDf.to_csv("data-frames-cleaned/" + "slotSettings" + '.csv', index=False)

Next, we can create the database.

In [959]:
import sqlite3

# Create a new SQLite database
conn = sqlite3.connect('../analysis/bioprinting.db')

# create sqlite tables
overviewDf.to_sql('overview', conn, if_exists='replace', index=False)
operatorsDf.to_sql('operators', conn, if_exists='replace', index=False)
slotSettingsDf.to_sql('slotSettings', conn, if_exists='replace', index=False)
bioInksDf.to_sql('bioInks', conn, if_exists='replace', index=False)
printingLogDf.to_sql('printingLog', conn, if_exists='replace', index=False)

908

# Test SQL queries

In [960]:
# SQL query to calculate the percentages
query = """
SELECT
  (SUM(CASE WHEN status = 'success' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS success_percentage,
  (SUM(CASE WHEN status = 'failed' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS failed_percentage,
  (SUM(CASE WHEN status NOT IN ('success', 'failed') THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS other_percentage
FROM
  printingLog;
"""

# Execute the SQL query
cursor = conn.cursor()
cursor.execute(query)

# Fetch the results
percentages = cursor.fetchone()

# Assign the percentages to variables
success_percentage, failed_percentage, other_percentage = percentages

# Print the results
print(f"Success percentage: {success_percentage}%")
print(f"Failed percentage: {failed_percentage}%")
print(f"Other statuses percentage: {other_percentage}%")

# Close the cursor and the connection
cursor.close()
conn.close()

Success percentage: 52.97356828193833%
Failed percentage: 43.392070484581495%
Other statuses percentage: 3.6343612334801763%
